# IS-RSA for networks

In [1]:
import os
os.chdir('/project/3013104.01/')
os.getcwd()

'/project/3013104.01'

## Compute inter-subject representation between inter-subject network activation and intersubject SPSQ score

In [2]:
import os
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from itertools import permutations
import datetime
import glob

# Function to create permuted distance matrices
def create_permuted_distance_matrices(original_distance_matrix, n_permutations):
    num_subjects = original_distance_matrix.shape[0]
    permuted_distance_matrices = []
    
    for _ in range(n_permutations):
        # Create a random permutation of the questionnaire scores
        permutation_indices = np.random.permutation(num_subjects)

        # Reconstruct the permuted distance matrix while preserving subject order
        permuted_matrix = original_distance_matrix[permutation_indices][:, permutation_indices]
        
        permuted_distance_matrices.append(permuted_matrix)
    
    return permuted_distance_matrices

# Function to concatenate CSV files
def concatenate_csv_files(frame, dimension, formatted_date):
    isrsa_dir = f'analysis/inter-subject_representational_similarity_analysis/{frame}_{formatted_date}_networks_{model}/{dimension}'
    out_dir = isrsa_dir

    concatenated_dataframes = []

    for csv_file in glob.glob(os.path.join(isrsa_dir, '*.csv')):
        df = pd.read_csv(csv_file)
        concatenated_dataframes.append(df)

    concatenated_dataframe = pd.concat(concatenated_dataframes, ignore_index=True)

    output_filename = os.path.join(out_dir, f'{dimension}_isrsa_concat.csv')
    concatenated_dataframe.to_csv(output_filename, index=False)

    print(f"Concatenated {dimension} in {frame}")

                          
aural_framings = ['neutral', 'threat']
models = ['euclidean_distance']
dimensions = ['SPSQ_pos', 'SPSQ_neg'] 

# Today's date
today = datetime.date.today()
formatted_date = today.strftime('%d-%m-%Y')

for frame in aural_framings:
    for model in models:
        for dimension in dimensions:

            fmri_similarity_dir = f'analysis/inter-subject_matrices_fmri/network/dissimilarity_matrices_mfmri_network_{frame}'
            output_dir = f'analysis/inter-subject_representational_similarity_analysis/{frame}_{formatted_date}_networks_{model}/{dimension}'

            # Delete output_dir if it exists
            if os.path.exists(output_dir):
                for file in os.listdir(output_dir):
                    file_path = os.path.join(output_dir, file)
                    try:
                        if os.path.isfile(file_path):
                            os.unlink(file_path)
                    except Exception as e:
                        print(e)
                os.rmdir(output_dir)

            # Create output_dir
            os.makedirs(output_dir, exist_ok=True)

            # List all brain network (dis)similarity files in fmri_similarity_dir
            brain_matrices = [f for f in os.listdir(fmri_similarity_dir) if f.endswith('.csv')]

            # Questionnaire distance/dissimilarity matrix 
            questionnaire_matrix_path = f'analysis/inter-subject_matrices_spsq/{model}_composite/{model}_matrix_{dimension}.csv'
            questionnaire_matrix = pd.read_csv(questionnaire_matrix_path, header=0, index_col=0).values
            questionnaire_vector = questionnaire_matrix[np.tril_indices(questionnaire_matrix.shape[0])]

            # How many permutations?
            n_permutations = 10000

            # Create a list of permuted distance matrices to use for all brain matrices
            permuted_distance_matrices = create_permuted_distance_matrices(questionnaire_matrix, n_permutations)

            # Loop through each brain node similarity file
            for brain_matrix_file in brain_matrices:
                print(f"Processing: {brain_matrix_file}")

                brain_matrix = pd.read_csv(os.path.join(fmri_similarity_dir, brain_matrix_file), header=None, index_col=None).values
                brain_vector = brain_matrix[np.tril_indices(brain_matrix.shape[0])]

                # Spearman correlation between vectorized lower triangles of brain and questionnaire matrices
                correlation, p_value = spearmanr(brain_vector, questionnaire_vector, nan_policy='omit')

                # Permutations using the pre-generated list of permuted distance matrices
                correlation_permuted = []
                for permuted_matrix in permuted_distance_matrices:
                    permuted_questionnaire_vector = permuted_matrix[np.tril_indices(permuted_matrix.shape[0])]
                    corr, _ = spearmanr(brain_vector, permuted_questionnaire_vector, nan_policy='omit')
                    correlation_permuted.append(corr)

                p_value_corrected = (np.sum(np.abs(correlation_permuted) >= np.abs(correlation)) + 1) / (n_permutations + 1)

                # Dataframe to store results
                result_df = pd.DataFrame({
                    'network_similarity_matrix': [brain_matrix_file],
                    'spearman_r': [correlation],
                    'p_uncorrected': [p_value],
                    'p_permuted': [p_value_corrected]
                })

                # Save in output_dir
                output_file = os.path.splitext(brain_matrix_file)[0] + '_isrsa.csv'
                result_df.to_csv(os.path.join(output_dir, output_file), index=False)

                print(f"Complete: {output_file}")

            # Call the function to concatenate CSV files
            concatenate_csv_files(frame, dimension, formatted_date)

Processing: dissimilarity_salience_ventral_attention.csv
Complete: dissimilarity_salience_ventral_attention_isrsa.csv
Processing: dissimilarity_control.csv
Complete: dissimilarity_control_isrsa.csv
Processing: dissimilarity_default.csv
Complete: dissimilarity_default_isrsa.csv
Concatenated SPSQ_pos in neutral
Processing: dissimilarity_salience_ventral_attention.csv
Complete: dissimilarity_salience_ventral_attention_isrsa.csv
Processing: dissimilarity_control.csv
Complete: dissimilarity_control_isrsa.csv
Processing: dissimilarity_default.csv
Complete: dissimilarity_default_isrsa.csv
Concatenated SPSQ_neg in neutral
Processing: dissimilarity_salience_ventral_attention.csv
Complete: dissimilarity_salience_ventral_attention_isrsa.csv
Processing: dissimilarity_control.csv
Complete: dissimilarity_control_isrsa.csv
Processing: dissimilarity_default.csv
Complete: dissimilarity_default_isrsa.csv
Concatenated SPSQ_pos in threat
Processing: dissimilarity_salience_ventral_attention.csv
Complete: d

## Concatenate for each model of SPS (dis)similarity

In [4]:
import os
import pandas as pd

framing = ['neutral', 'threat']
models = ['euclidean_distance']
dimensions = ['SPSQ_pos', 'SPSQ_neg']

for model in models:
    # Initialize an empty list to store DataFrames
    data_frames = []
    
    for frame in framing:
        for dim in dimensions:
            working_dir = f'analysis/inter-subject_representational_similarity_analysis/{frame}_03-01-2024_networks_{model}/{dim}'
            file_path = f'{working_dir}/{dim}_isrsa_concat.csv'
            
            # Check if the file exists before reading it
            if os.path.exists(file_path):
                data = pd.read_csv(file_path)
                # Add "framing" and "dimension" columns
                data['framing'] = frame
                data['dimension'] = dim
                data_frames.append(data)

    # Concatenate the list of DataFrames into one DataFrame
    concatenated_data = pd.concat(data_frames, ignore_index=True)

    # Save the concatenated data to the out_file
    out_file = f'analysis/inter-subject_representational_similarity_analysis/{model}_results_concat.csv'
    concatenated_data.to_csv(out_file, index=False)


## FDR correction

In [5]:
import pandas as pd
from statsmodels.stats.multitest import fdrcorrection

# Load your CSV file into a DataFrame
file_path = "analysis/inter-subject_representational_similarity_analysis/euclidean_distance_results_concat.csv"
df = pd.read_csv(file_path)

# Perform FDR correction
pvals_corrected = fdrcorrection(df['p_permuted'])[1]

# Create a new column in the DataFrame for the corrected p-values
df['p_fdr'] = pvals_corrected

# Save the updated DataFrame back to a CSV file
output_file = "analysis/inter-subject_representational_similarity_analysis/euclidean_distance_results_concat_subset.csv"
df.to_csv(output_file, index=False)

print("FDR correction completed and saved to", output_file)


FDR correction completed and saved to analysis/inter-subject_representational_similarity_analysis/euclidean_distance_results_concat_subset.csv


## Plot IS-RSA results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize

# Load data from CSV file
file_path = 'analysis/inter-subject_representational_similarity_analysis/euclidean_distance_results_concat_subset.csv'
df = pd.read_csv(file_path, header=0, index_col=None)

# Sort data by 'SPSQ dimension', 'functional network', and 'aural framing'
df.sort_values(by=['SPSQ dimension', 'functional network', 'aural framing'], inplace=True)

# Separate data by network and aural framing
networks = sorted(set(df['functional network']))
aural_framings = sorted(set(df['aural framing']))
dimensions = sorted(set(df['SPSQ dimension']))

# Define colors for each network
network_colors = {'control': '#DA963F', 'default': '#BC4751', 'salience': '#B345EE'}

# Set font size
font_size = 26

# Plotting
fig, axs = plt.subplots(len(dimensions), 1, figsize=(8, 11), sharex=True, gridspec_kw={'hspace': 0.4})

# Initialize variables to store maximum and minimum y-axis values
max_y = float('-inf')
min_y = float('inf')

for i, dimension in enumerate(dimensions):
    ax = axs[i]

    tick_labels = []  # List to store tick labels
    tick_positions = []  # List to store tick positions
    
    for j, network in enumerate(networks):
        bars = []  # List to store bar objects for legend
        for k, framing in enumerate(aural_framings):
            # Filter data for the current SPSQ dimension, network, and aural framing
            subset = df[(df['SPSQ dimension'] == dimension) & (df['functional network'] == network) & (df['aural framing'] == framing)]

            # Extract values
            rs = subset['Spearman r'].values
            positions = np.arange(len(rs)) + (k + j * len(aural_framings)) * 0.4  # Adjust the spacing (0.4) as needed

            tick_positions.extend(positions)  # Add positions to the list
            tick_labels.extend([f'{network} - {framing}' for _ in positions])  # Add labels to the list

            # Update maximum and minimum y-axis values
            max_y = max(max_y, np.max(rs))
            min_y = min(min_y, np.min(rs))

            # Define hatch pattern based on 'aural framing'
            hatch = None
            if framing == 'threat':
                hatch = '////'  # Pattern for 'threat' framing

            # Plotting bars with network-specific color and label for aural framing
            bar = ax.bar(positions, rs, label=f'{network} - {framing}', width=0.35, 
                         color=network_colors[network], edgecolor='black', linewidth=1, alpha=0.9, hatch=hatch)
            bars.extend(bar)

    ax.set_title(f'SPSQ-SF {dimension} dimension', fontsize=font_size)

    # Set y-axis label for the first subplot
    #if i == 0 and dimension != 'positive':
        #set_ylabel('IS-RSA Spearman r', fontsize=font_size)

# Set y-axis limits for all subplots
for ax in axs:
    ax.set_ylim(0, .2)

# Replace specific strings in the x-axis tick labels
tick_labels = [label.replace('control -', 'CEN').replace('default -', 'DMN').replace('salience -', 'SN') for label in tick_labels]

# Set x-axis tick positions and labels for the bottom subplot
axs[-1].set_xticks(tick_positions)
axs[-1].set_xticklabels(tick_labels, rotation=45, ha='right')
axs[-1].set_xlabel('network and condition', fontsize=font_size)

# Remove legend for the negative dimension plot
axs[0].legend().set_visible(False)

# Remove box around plots
for ax in axs:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# Increase tick font size
for ax in axs:
    ax.tick_params(axis='both', which='major', labelsize=font_size)

# Add a single y-axis label spanning the length of both plots
fig.text(0.05, 0.63, 'IS-RSA Spearman r\n(neural dissimilarity ~ SPSQ-SF distance)', 
         va='center', rotation='vertical', fontsize=font_size, ha='center')

# Adjust the figure boundaries to ensure the y-axis title is fully visible
plt.subplots_adjust(left=0.25, right=0.95, top=0.95, bottom=0.4)

# Save the entire figure as an A4 portrait PDF
pdf_path = 'analysis/plots/isrsa_network_spearman_r_bars.pdf'
plt.savefig(pdf_path, format='pdf')
plt.show()
